In [54]:
import numpy as np
import copy

In [55]:
#generate random bipartite graphs with complete preference list
def rand_matching(n):
    A_pref = {} # keys = men, values = list of preference of key on women
    B_pref = {} # keys = women, values = list of preference of key on men
    for i in range(n):
        #preference list for every member in A
        pref_a = list(np.random.permutation(n))
        pref_a = ['B'+ str(x) for x in pref_a]
        A_pref['A' + str(i)] = pref_a

        #preference list for every member in B
        pref_b = list(np.random.permutation(n))
        pref_b = ['A'+ str(x) for x in pref_b]
        B_pref['B' + str(i)] = pref_b
    return A_pref, B_pref

In [68]:
Aprefers, Bprefers = rand_matching(50)
# print(Aprefers)
# print(Bprefers)

In [69]:
#executes gale_shapley
#input is A and B's preference lists
#outputs stable matching and the last a and b that was matched (used for Gale Shapley extended)
#note that this assumes that |A|=|B| and both sets have complete preference lists
#suppose not, one should implement different escape conditions - ie, stop trying to match current_a if its preference list becomes empty
def gale_shapley(A_pref, B_pref):
    A_pref2 = copy.deepcopy(A_pref) # need this because later we pop the preference lists stored in A_pref2, B_pref2
    B_pref2 = copy.deepcopy(B_pref)
    A = sorted(A_pref.keys()) # A = ['A0', 'A1', ..., 'A{n-1}']
    B = sorted(B_pref.keys()) # B = ['B0', 'B1', ..., 'B{n-1}']
    A_free = A[:] # = A initially
    A_matched  = {} # keys = women (B), values = currently matched men (A)
    while A_free: # while there are still unmatched men
        current_a = A_free.pop(0)
        #get current node's preference list
        pref_a = A_pref2[current_a]

        not_matched = 1
        while not_matched:
            partner = pref_a.pop(0) # modified pref_a since pop(0) will remove the first element in the preference list
            match = A_matched.get(partner) # get() returns None if partner is not yet in A_matched
            if not match: # i.e., if match == None
            # if not yet matched
                A_matched[partner] = current_a # match current_a with partner
                #stores last pair matched
                last_a = current_a
                last_b = partner
                not_matched = 0
            else: # if partner is matched to some man already
            # if potential partner already matched, check preference list
                pref_b = B_pref2[partner]
                if pref_b.index(match) > pref_b.index(current_a):
                    # prefers current_a
                    A_matched[partner] = current_a
                    not_matched = 0
                    # no need to update last pair matched, because the original match will be matched later
                    # add original match to A_free list to be matched again
                    A_free.append(match)

    return A_matched, last_a, last_b

In [70]:
A_matched, last_a, last_b = gale_shapley(Aprefers, Bprefers)
print(A_matched)
print(last_a)
print(last_b)

{'B31': 'A25', 'B1': 'A1', 'B37': 'A10', 'B24': 'A11', 'B5': 'A30', 'B17': 'A13', 'B20': 'A35', 'B41': 'A15', 'B4': 'A17', 'B16': 'A43', 'B3': 'A18', 'B19': 'A47', 'B39': 'A26', 'B25': 'A45', 'B26': 'A21', 'B0': 'A22', 'B11': 'A41', 'B42': 'A44', 'B36': 'A16', 'B15': 'A27', 'B45': 'A46', 'B47': 'A37', 'B35': 'A3', 'B38': 'A31', 'B28': 'A32', 'B21': 'A40', 'B7': 'A0', 'B8': 'A24', 'B44': 'A48', 'B30': 'A4', 'B29': 'A20', 'B12': 'A14', 'B48': 'A36', 'B2': 'A7', 'B34': 'A42', 'B33': 'A23', 'B18': 'A33', 'B13': 'A2', 'B23': 'A49', 'B40': 'A29', 'B46': 'A9', 'B32': 'A6', 'B27': 'A19', 'B43': 'A5', 'B6': 'A39', 'B14': 'A34', 'B9': 'A12', 'B22': 'A38', 'B49': 'A28', 'B10': 'A8'}
A8
B10


In [71]:
#checks if there are blocking pairs for any matching (whether the input matching is stable)
#takes input matching, A's preference list, and B's preference list
def check(final_match, Aprefers, Bprefers):
    blocking_pairs = 0
    inverse_match = dict((v,k) for k,v in final_match.items()) # keys in final_match are women (B), in inverse_match are men (A)
    for b, a in final_match.items():
        #for every pair (a,b) in matching, take all that a prefers over b, and all that b prefers over a
        #these are potential blocking pairs
        bpref = Bprefers[b]
        b_block = bpref[:bpref.index(a)] # b_block contains all the men (A's) that b prefers to her current match a
        apref = Aprefers[a]
        a_block = apref[:apref.index(b)] # a_block contains all the women (B's) that a prefers to his current match b
        #check all of their matchings to see if they all prefer their matching over b
        for a1 in b_block: # for every man (a1) that b prefers to the current match a
            a1_b = inverse_match[a1] # a1_b is the woman matched to a1 in the current matching
            a1_pref = Aprefers[a1]
            if a1_pref.index(a1_b) > a1_pref.index(b): # this means a1 prefers b to his current matching
                print("%s and %s like each other better than "
                      "their present partners: %s and %s, respectively"
                      % (b, a1, a, a1_b))
                blocking_pairs += 1

        for b1 in a_block:
            b1_a = final_match[b1]
            b1_pref = Bprefers[b1]
            if b1_pref.index(b1_a) > b1_pref.index(a):
                print("%s and %s like each other better than "
                      "their present partners: %s and %s, respectively"
                      % (a, b1, b, b1_a))
                blocking_pairs += 1
    print("total number of blocking pairs = %d" % blocking_pairs)

In [72]:
check(A_matched, Aprefers, Bprefers)

total number of blocking pairs = 0


In [61]:
#gale_shapley extended; takes A's and B's preference lists, outputs Pareto-optimal matching (stable/unstable)
def gale_shapley_extended(A_pref, B_pref):
    #initial run of gale shapley
    A_unfixed = sorted(A_pref.keys())
    B_unfixed = sorted(B_pref.keys())
    A_pref2 = copy.deepcopy(A_pref)
    B_pref2 = copy.deepcopy(B_pref)
    A_matched, last_a, last_b = gale_shapley(A_pref2, B_pref2)
    n = len(A_matched)
    fixed_matches = {}
    for i in range(n-1):
        #fix the match of last_a and last_b
        A_unfixed.remove(last_a)
        B_unfixed.remove(last_b)
        fixed_matches[last_b] = last_a
        #remove last_a and last_b from preference list
        del A_pref2[last_a]
        del B_pref2[last_b]
        #remove last_a and last_b from everyone else's preference list
        for j in A_unfixed:
            A_pref2[j].remove(last_b)
        for j in B_unfixed:
            B_pref2[j].remove(last_a)
        #run Gale Shapley again
        A_matched, last_a, last_b = gale_shapley(A_pref2, B_pref2)

    A_matched.update(fixed_matches)

    return A_matched

In [73]:
final_match = gale_shapley_extended(Aprefers,Bprefers)
print(final_match)

{'B31': 'A0', 'B10': 'A8', 'B49': 'A28', 'B22': 'A38', 'B32': 'A19', 'B45': 'A23', 'B9': 'A12', 'B46': 'A9', 'B7': 'A34', 'B6': 'A26', 'B19': 'A47', 'B30': 'A44', 'B33': 'A39', 'B29': 'A20', 'B44': 'A36', 'B23': 'A5', 'B14': 'A35', 'B36': 'A16', 'B8': 'A24', 'B4': 'A2', 'B43': 'A17', 'B11': 'A41', 'B27': 'A46', 'B16': 'A43', 'B2': 'A7', 'B40': 'A6', 'B13': 'A49', 'B18': 'A48', 'B25': 'A45', 'B34': 'A42', 'B48': 'A40', 'B12': 'A4', 'B39': 'A37', 'B21': 'A33', 'B28': 'A32', 'B38': 'A31', 'B5': 'A30', 'B35': 'A3', 'B47': 'A29', 'B15': 'A27', 'B42': 'A25', 'B0': 'A22', 'B26': 'A21', 'B3': 'A18', 'B41': 'A15', 'B20': 'A14', 'B17': 'A13', 'B24': 'A11', 'B37': 'A10', 'B1': 'A1'}


In [74]:
#compare number of members of A that prefers gale_shapley extended matching over gale shapley matching
#takes input A and B's preference list, Gale_shapley matching, and galey shapley extended matching
def compare(A_pref, B_pref, A_matched, final_match):
    A = sorted(A_pref.keys())
    improved_A = 0
    bad_A = 0
    for i in A:
        pref_a = A_pref[i]
        match_new = list(final_match.keys())[list(final_match.values()).index(i)] # woman matched to man i in GSE
        match_old = list(A_matched.keys())[list(A_matched.values()).index(i)] # woman matched to man i in GS
        if pref_a.index(match_new) < pref_a.index(match_old): # women in GSE is preferred
            improved_A += 1
        elif pref_a.index(match_new) > pref_a.index(match_old):
            bad_A += 1

    print("%d members of A are doing better than before" % improved_A)
    print("%d members of A are doing worse than before" % bad_A)

In [75]:
compare(Aprefers, Bprefers, A_matched, final_match)

23 members of A are doing better than before
0 members of A are doing worse than before


In [76]:
check(final_match, Aprefers, Bprefers)

A8 and B4 like each other better than their present partners: B10 and A2, respectively
A28 and B45 like each other better than their present partners: B49 and A23, respectively
A38 and B30 like each other better than their present partners: B22 and A44, respectively
A19 and B30 like each other better than their present partners: B32 and A44, respectively
B45 and A28 like each other better than their present partners: A23 and B49, respectively
A23 and B40 like each other better than their present partners: B45 and A6, respectively
A12 and B18 like each other better than their present partners: B9 and A48, respectively
B30 and A38 like each other better than their present partners: A44 and B22, respectively
B30 and A19 like each other better than their present partners: A44 and B32, respectively
B4 and A8 like each other better than their present partners: A2 and B10, respectively
B40 and A23 like each other better than their present partners: A6 and B45, respectively
B18 and A12 like ea